In [1]:
# Import the dependencies

# Import numpy for mathematical functions and values
import numpy as np

# Import the citypy library to use a function to obtain the nearest city; note that citipy is installed in our PythonData environment
from citipy import citipy

# Import the API key that is required for OpenWeather API requests; the config.py file is a private file that is not within the GitHub repository
from config import weather_api_key

# Import the requests library for connecting with API endpoints
import requests

# Import time library for staggering the API requests (to avoid overloading the API endpoint with data requests)
import time

# Import pandas for ETL processes
import pandas as pd

# Import Matplotlib for graphing
import matplotlib.pyplot as plt

In [2]:
# Create a set of random latitude and longitude pairs using the np.random function
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Pair the latitudes and longitudes values together
lat_lngs = zip(lats, lngs)

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
# Create a list for holding the cities information that will be displayed on a map
cities = []

# Identify the nearest city for each latitude and longitude combination that we randomly generated above
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

In [5]:
# Define the OpenWeather API endpoint URL
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Define an empty list to hold the retrieved weather data
city_data = []

# Define a record counter for keeping track of how many city records have been retrieved within a set
record_count = 1

# Define a set counter for keeping track of the set of city records that have been retrieved from the API call
# In other words, each set corresponds to a custer of API GET requests for a cluster of cities
set_count = 1

# Print the beginning of the logging process
print("-----------------------------")
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Loop through all the cities in the cities list using the built-in enumerate function
for i, city in enumerate(cities):

    # Group cities in sets of 50 to avoid overloading the OpenWeather API endpoint with GET requests
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an API endpoint URL with a city embedded within it
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the current record count, set count, and city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Increment the record counter
    record_count += 1
    
    # Run an API GET request for each of the cities in the cities array
    try:
        # Retrieve and parse the JSON data from the OpenWeather API GET request
        city_weather = requests.get(city_url).json()

        # Extract the desired data from the retrieved weather JSON data
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather["weather"][0]["description"]

        # Append a city's weather information to the city_data list
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# In case there is an error in retrieving a city's weather data, skip that city
    except:
        print("City not found. Skipping...")
        pass
    
# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

-----------------------------
Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | ushuaia
Processing Record 2 of Set 1 | krasnoselkup
Processing Record 3 of Set 1 | kendari
Processing Record 4 of Set 1 | dali
Processing Record 5 of Set 1 | obo
Processing Record 6 of Set 1 | jamestown
Processing Record 7 of Set 1 | ahipara
Processing Record 8 of Set 1 | saldanha
Processing Record 9 of Set 1 | busselton
Processing Record 10 of Set 1 | nelson bay
Processing Record 11 of Set 1 | illoqqortoormiut
City not found. Skipping...
Processing Record 12 of Set 1 | iskateley
Processing Record 13 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 14 of Set 1 | atuona
Processing Record 15 of Set 1 | thompson
Processing Record 16 of Set 1 | ponta do sol
Processing Record 17 of Set 1 | kondopoga
Processing Record 18 of Set 1 | mar del plata
Processing Record 19 of Set 1 | vaini
Processing Record 20 of Set 1 | cabo san lucas
Processing Re

Processing Record 36 of Set 4 | raudeberg
Processing Record 37 of Set 4 | banikoara
Processing Record 38 of Set 4 | ostrovnoy
Processing Record 39 of Set 4 | hithadhoo
Processing Record 40 of Set 4 | murfreesboro
Processing Record 41 of Set 4 | iqaluit
Processing Record 42 of Set 4 | barentsburg
City not found. Skipping...
Processing Record 43 of Set 4 | belen
Processing Record 44 of Set 4 | severo-kurilsk
Processing Record 45 of Set 4 | carbondale
Processing Record 46 of Set 4 | padang
Processing Record 47 of Set 4 | kupang
Processing Record 48 of Set 4 | mega piston
City not found. Skipping...
Processing Record 49 of Set 4 | innisfail
Processing Record 50 of Set 4 | hokitika
Processing Record 1 of Set 5 | mys shmidta
City not found. Skipping...
Processing Record 2 of Set 5 | port elizabeth
Processing Record 3 of Set 5 | nam tha
City not found. Skipping...
Processing Record 4 of Set 5 | dongsheng
Processing Record 5 of Set 5 | rocha
Processing Record 6 of Set 5 | voh
Processing Record

Processing Record 20 of Set 8 | mogadishu
Processing Record 21 of Set 8 | khanpur
Processing Record 22 of Set 8 | dalby
Processing Record 23 of Set 8 | sarangani
Processing Record 24 of Set 8 | paciran
Processing Record 25 of Set 8 | aktas
Processing Record 26 of Set 8 | kuldur
Processing Record 27 of Set 8 | warwick
Processing Record 28 of Set 8 | shizilu
Processing Record 29 of Set 8 | novopavlovka
Processing Record 30 of Set 8 | nome
Processing Record 31 of Set 8 | san policarpo
Processing Record 32 of Set 8 | mount gambier
Processing Record 33 of Set 8 | mbandaka
Processing Record 34 of Set 8 | scottsbluff
Processing Record 35 of Set 8 | penzance
Processing Record 36 of Set 8 | moree
Processing Record 37 of Set 8 | mukhen
Processing Record 38 of Set 8 | urubicha
Processing Record 39 of Set 8 | lensk
Processing Record 40 of Set 8 | sungairaya
Processing Record 41 of Set 8 | pishin
Processing Record 42 of Set 8 | vao
Processing Record 43 of Set 8 | henties bay
Processing Record 44 of

Processing Record 12 of Set 12 | mairi
Processing Record 13 of Set 12 | enkoping
Processing Record 14 of Set 12 | gamba
Processing Record 15 of Set 12 | kruisfontein
Processing Record 16 of Set 12 | makung
City not found. Skipping...
Processing Record 17 of Set 12 | inhambane
Processing Record 18 of Set 12 | pundaguitan
Processing Record 19 of Set 12 | tucuman
Processing Record 20 of Set 12 | artyk
City not found. Skipping...
Processing Record 21 of Set 12 | quatre cocos
Processing Record 22 of Set 12 | kyaikto
Processing Record 23 of Set 12 | pokrovsk
Processing Record 24 of Set 12 | biltine
Processing Record 25 of Set 12 | asau
Processing Record 26 of Set 12 | anchorage
Processing Record 27 of Set 12 | ambilobe
Processing Record 28 of Set 12 | sfantu gheorghe
Processing Record 29 of Set 12 | puerto ayacucho
Processing Record 30 of Set 12 | caxito
Processing Record 31 of Set 12 | margate
Processing Record 32 of Set 12 | mae sai
Processing Record 33 of Set 12 | kungurtug
Processing Rec

Processing Record 47 of Set 15 | markova
Processing Record 48 of Set 15 | ijaki
City not found. Skipping...
Processing Record 49 of Set 15 | hihifo
City not found. Skipping...
Processing Record 50 of Set 15 | jurm
Processing Record 1 of Set 16 | vestmannaeyjar
Processing Record 2 of Set 16 | beaufort
Processing Record 3 of Set 16 | urumqi
Processing Record 4 of Set 16 | castanos
Processing Record 5 of Set 16 | novyy urengoy
Processing Record 6 of Set 16 | luba
Processing Record 7 of Set 16 | ipoh
Processing Record 8 of Set 16 | taft
Processing Record 9 of Set 16 | west helena
Processing Record 10 of Set 16 | kobojango
City not found. Skipping...
Processing Record 11 of Set 16 | ayame
Processing Record 12 of Set 16 | hualmay
Processing Record 13 of Set 16 | chimore
Processing Record 14 of Set 16 | bandarbeyla
Processing Record 15 of Set 16 | gelibolu
-----------------------------
Data Retrieval Complete      
-----------------------------


In [7]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

# Check to see if the data was properly loaded
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,42.46,52,75,16.11,broken clouds
1,Krasnoselkup,RU,65.7000,82.4667,31.48,83,100,12.55,overcast clouds
2,Kendari,ID,-3.9450,122.4989,76.59,85,99,2.37,overcast clouds
3,Dali,CN,25.7000,100.1833,61.83,90,66,3.04,broken clouds
4,Obo,CF,5.3998,26.4869,86.16,57,76,2.53,broken clouds
5,Jamestown,US,42.0970,-79.2353,54.72,60,100,8.05,mist
6,Ahipara,NZ,-35.1667,173.1667,54.81,92,65,5.79,broken clouds
7,Saldanha,ZA,-33.0117,17.9442,68.05,67,31,11.86,scattered clouds
8,Busselton,AU,-33.6500,115.3333,52.88,84,34,8.68,scattered clouds
9,Nelson Bay,AU,-32.7167,152.1500,61.00,94,16,7.00,few clouds


In [8]:
# Create and output a .csv file

# Define the output file name and its directory
output_data_file = "weatherpy_database.csv"

# Export the city_data_df to the the previously defined .csv file
city_data_df.to_csv(output_data_file, index_label = "City_ID")